# test DL models

## so much import

In [1]:
import os
import sys
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

In [15]:
from importlib import reload

In [12]:
top_path = os.path.dirname(os.path.abspath("./"))
sys.path.append(top_path)
print(top_path)

/home/wergillius/Project/UTR_VAE


In [19]:
import utils
reload(utils)

/home/wergillius/Project/UTR_VAE
/home/wergillius/Project/UTR_VAE


<module 'utils' from '/home/wergillius/Project/UTR_VAE/utils.py'>

In [21]:
from model import DL_models
from model import reader

### read data

In [23]:
# define data
dataset = reader.UTR_dataset(cell_line='A549')

train_loader,val_loader,test_loader = reader.get_splited_dataloader(dataset,
                                                                    ratio=[0.7,0.2,0.1],
                                                                    batch_size=10,
                                                                    num_workers=4)

In [27]:
train_iter=iter(train_loader)

In [28]:
# from train loader get some data
batch_data = next(train_iter)

X, y = batch_data
X = X.float()
y = y.long()

In [37]:
X[0]

tensor([[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0.,

In [29]:
X.shape , y.shape

(torch.Size([10, 100, 4]), torch.Size([10]))

## Test DL models

In [32]:
# define some parameters for LSTM

laten_dim = 32
hidden_size = 8
out_dim = 2*hidden_size*X.shape[1]

In [31]:
L_encoder = DL_models.LSTM_backbond(input_size=4,
                                    hidden_size=8,
                                    num_layers=3,
                                    bidirectional=True)

In [34]:
L_decoder = DL_models.LSTM_backbond(input_size=laten_dim,
                                    hidden_size=4,
                                    num_layers=3,
                                    bidirectional=False)

In [35]:
model = DL_models.VAE(L_encoder,L_decoder,latent_dim=laten_dim,out_dim=out_dim)

In [36]:
one_repeat_out = model(X)

TypeError: can't multiply sequence by non-int of type 'float'

In [41]:
X = torch.randint(10,(20,100,4))

In [68]:
max_num = X.max(dim=2)

In [ ]:
torch.broadcast_tensors()

In [76]:
max_num.

torch.return_types.max(
values=tensor([[6, 7, 4,  ..., 8, 7, 9],
        [5, 7, 9,  ..., 8, 9, 5],
        [6, 8, 7,  ..., 5, 9, 8],
        ...,
        [9, 6, 5,  ..., 8, 9, 5],
        [5, 9, 7,  ..., 8, 6, 9],
        [9, 8, 5,  ..., 7, 9, 4]]),
indices=tensor([[0, 2, 3,  ..., 0, 0, 1],
        [2, 2, 2,  ..., 3, 3, 1],
        [0, 0, 2,  ..., 0, 0, 0],
        ...,
        [0, 2, 1,  ..., 0, 3, 0],
        [2, 1, 0,  ..., 0, 1, 1],
        [2, 0, 2,  ..., 1, 2, 2]]))

In [72]:
X == max_num[0]

RuntimeError: The size of tensor a (4) must match the size of tensor b (100) at non-singleton dimension 2

In [47]:
position = X.argmax(dim=2)

In [61]:
seq = torch.zeros_like(X)

In [80]:
for batch in range(X.shape[0]):
    for i,j in enumerate(position[batch]):
        X[batch,i,j.item()] = 1

In [81]:
X[0,:10,:]

tensor([[1, 2, 0, 2],
        [0, 4, 1, 1],
        [3, 2, 1, 1],
        [1, 6, 4, 6],
        [0, 3, 5, 1],
        [1, 4, 5, 7],
        [5, 6, 1, 4],
        [5, 4, 6, 1],
        [0, 2, 3, 1],
        [1, 5, 6, 6]])

In [84]:
seq2 = seq.clone()

In [67]:
seq[0,:10,:]

tensor([[1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [1, 0, 0, 0]])

In [86]:
sim = seq.mul(seq2)

In [89]:
sim.sum(dim=2).sum(dim=1)

tensor([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100])

In [90]:
import logging

In [92]:
logging.warning('careful !')

In [93]:
logging.info("wow")

In [91]:
logging.debug('This message should go to the log file')

In [ ]:
logger = logging.